<a href="https://colab.research.google.com/github/Paunty/SeaWex/blob/main/tutorial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Train YOLOv5 on Custom Objects for Seeed Vision product

This tutorial is based on the [YOLOv5-swift repository](https://github.com/Seeed-Studio/yolov5-swift) by [Seeed-Studio](https://www.seeedstudio.com/). This notebook shows training on **your own custom objects**. Many thanks to USeeed-Studio for putting this repository together - we hope that in combination with clean data management tools at Roboflow, this technologoy will become easily accessible to any developer wishing to use computer vision in their projects.

### Accompanying Blog Post

We recommend that you follow along in this notebook while reading the blog post on [how to train YOLOv5](https://blog.roboflow.ai/how-to-train-yolov5-on-a-custom-dataset/), concurrently.

### Steps Covered in this Tutorial

In this tutorial, we will walk through the steps required to train YOLOv5 on your custom objects. We use a [public blood cell detection dataset](https://public.roboflow.ai/object-detection/bccd), which is open source and free to use. You can also use this notebook on your own data.

To train our detector we take the following steps:

* Install YOLOv5 dependencies
* Download custom YOLOv5 object detection data
* Write our YOLOv5 Training configuration
* Run YOLOv5 training
* Evaluate YOLOv5 performance
* Visualize YOLOv5 training data
* Run YOLOv5 inference on test images
* Export saved YOLOv5 weights for future inference



### **About**

[Roboflow](https://roboflow.com) enables teams to deploy custom computer vision models quickly and accurately. Convert data from to annotation format, assess dataset health, preprocess, augment, and more. It's free for your first 1000 source images.

**Looking for a vision model available via API without hassle? Try Roboflow Train.**

![Roboflow Wordmark](https://files.seeedstudio.com/wiki/SenseCAP-A1101/57.png)



# Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [1]:
# clone YOLOv5 repository
%cd /content/
!git clone https://github.com/Seeed-Studio/yolov5-swift  # clone repo
%cd yolov5-swift

/content
Cloning into 'yolov5-swift'...
remote: Enumerating objects: 9409, done.
remote: Total 9409 (delta 0), reused 0 (delta 0), pack-reused 9409 (from 1)
Receiving objects: 100% (9409/9409), 22.67 MiB | 24.80 MiB/s, done.
Resolving deltas: 100% (6271/6271), done.
/content/yolov5-swift


In [2]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch
from IPython.display import Image, clear_output  # to display images
# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

ERROR: Could not find a version that satisfies the requirement tensorflow==2.9.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.9.0
Setup complete. Using torch 2.5.1+cu124 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15095MB, multi_processor_count=40, uuid=a87a237b-9581-7aec-90db-ed0acd9587bf, L2_cache_size=4MB)


# Get Correctly Formatted Custom Dataset [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/yolov5-custom-training-tutorial/blob/main/yolov5-custom-training.ipynb)
We'll download our dataset from Roboflow. Use the "**YOLOv5 PyTorch**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.

To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).
<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="480" src="https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/6152a275ad4b4ac20cd2e21a_roboflow-annotate.gif"/></a></p>Label images lightning fast (including with model-assisted labeling)

In [3]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolo5")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 10.6 MB/s eta 0:00:00


RuntimeError: API Key is of Incorrect Type 
 Expected Type: <class 'str'>
 Input Type: <class 'NoneType'>

In [4]:
%cd /content/yolov5-swift
#after following the link above, recieve python code with these fields filled in
from roboflow import Roboflow
rf = Roboflow(api_key="muvFLVF2ovd8ao6oIXud")
rf.workspace('det').list_projects()
project = rf.workspace('det').project("pascal-voc-2012-7wvwq")
dataset = project.version("2").download("yolov5")

/content/yolov5-swift
loading Roboflow workspace...
[{'id': 'det/bccd-digjp', 'type': 'object-detection', 'name': 'BCCD', 'created': 0, 'updated': 1650262922.566, 'images': 364, 'unannotated': 0, 'annotation': 'cells', 'versions': 1839, 'public': False, 'multilabel': False, 'license': 'MIT', 'splits': {'valid': 73, 'test': 36, 'train': 255}, 'colors': {'RBC': '#C7FC00', 'WBC': '#FF00FF', 'Platelets': '#8622FF'}, 'classes': {'RBC': 4155, 'WBC': 372, 'Platelets': 361}, 'icon': {'original': 'https://source.roboflow.com/QrgxvOuwBnM1tpZeMZeokOZ0hHV2/Wdt9c0nrwkegk8m9TxoL/original.jpg', 'thumb': 'https://source.roboflow.com/QrgxvOuwBnM1tpZeMZeokOZ0hHV2/Wdt9c0nrwkegk8m9TxoL/thumb.jpg', 'annotation': 'https://source.roboflow.com/QrgxvOuwBnM1tpZeMZeokOZ0hHV2/Wdt9c0nrwkegk8m9TxoL/annotation-cells.png'}, 'preprocessing': {'resize': {'width': '416', 'format': 'Stretch to', 'height': '416'}, 'remap': {'labels': {'RBC': {'omit': True}, 'WBC': {'omit': True}}}}, 'augmentation': {'saturation': {'percen

BadZipFile: File is not a zip file

In [6]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

cat: {dataset.location}/data.yaml: No such file or directory


# Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

You do not need to edit these cells, but you may.

In [5]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

NameError: name 'dataset' is not defined

download the pre-train model

In [ ]:

%cd /content/yolov5-swift/
!wget https://github.com/Seeed-Studio/yolov5-swift/releases/download/v0.1.0-alpha/yolov5n6-xiao.pt

# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5-swift/
!python train.py --img 192 --batch 64 --epochs 50 --data {dataset.location}/data.yaml --cfg yolov5n6-xiao.yaml --weights 'yolov5n6-xiao.pt' --name yolov5n6_results  --cache

# Evaluate Custom YOLOv5 Detector Performance

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason...
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5-swift/runs/train/yolov5n6_results/results.png', width=1000)  # view results.png

### Curious? Visualize Our Training Data with Labels

After training starts, view `train*.jpg` images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5-swift/runs/train/yolov5n6_results/val_batch0_labels.jpg', width=900)

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5-swift/runs/train/yolov5n6_results/val_batch0_pred.jpg', width=900)

# Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls runs/train/yolov5n6_results/weights

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/yolov5-swift/
!python detect.py --weights runs/train/yolov5n6_results/weights/best.pt --img 320 --conf 0.6 --source ./data/images --name test

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5-swift/runs/detect/test/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Export tflite file

In [ ]:
%cd /content/yolov5-swift/
!python export.py --data {dataset.location}/data.yaml --weights runs/train/yolov5n6_results/weights/best.pt --imgsz 192 --int8 --include tflite

# tflite to uf2

# GROVEAI For Grove Vision AI and Vision AI for SenseCAP A1101
# !python uf2conv.py -f GROVEAI -t 1 -c runs//train/yolov5n6_results//weights/best-int8.tflite -o best.uf2
# !python uf2conv.py -f VISIONAI  -t 1 -c runs//train/yolov5n6_results//weights/best-int8.tflite -o best.uf2

In [ ]:

!python uf2conv.py -f VISIONAI  -t 1 -c runs//train/yolov5n6_results//weights/best-int8.tflite -o best.uf2
%cp best.uf2 ../

In [7]:
from google.colab import files
uploaded = files.upload()


Saving boat_classifier.tflite to boat_classifier.tflite


In [9]:
# Ejecuta el comando de conversión usando los mismos parámetros que indicas
!python uf2conv.py -f GROVEAI -t 1 -c boat_classifier.tflite -o best.uf2

# Copia el archivo generado al directorio superior (si lo necesitas)
%cp best.uf2 ../

# Descarga el archivo UF2 al computador local
from google.colab import files
files.download('../best.uf2')

Converted to uf2, output size: 6772736, start address: 0x30000000
Wrote 6772736 bytes to best.uf2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Download Trained model for Future Inference

Now that you have trained your custom detector, you can export the trained model you have made here for inference on your device elsewhere

In [ ]:
print("Download your custom model")

# Please download the trained weight files in the following order
- Find the file option in the menu to the right of the colab
  
  ![download_1.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/download_1.png?raw=true)

- Select the .uf2 file generated above
  
  ![download_2.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/download_2.png?raw=true)

- Click the right mouse button and select Download
  
  ![download_3.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/download_3.png?raw=true)
  
  



# Deploy Your Model to Grove AI Devices
- Install the latest version of [Google Chrome](https://www.google.com/chrome/) or [Microsoft Edge](https://www.microsoft.com/edge) browser, and open it.

- Plug your Grove AI device into your pc.
- Download Model into your Grove AI device
  - Double-click the boot button

    You can see a GROVEBOOT label on your pc that can move the drive.
    ![deploy_1.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/deploy_1.png?raw=true)

  - Drag and drop the .uf2 file you downloaded to your GROVEBOOT removable disk
  - Open [grove ai vision preview](https://files.seeedstudio.com/grove_ai_vision/index.html)
    - Click Connect Button and select Grove AI Camera.
    ![deploy_2.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/deploy_2.png?raw=true)

  - View real-time speculative results through the preview window
    ![deploy_3.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/deploy_3.png?raw=true)

# Congrats!

Hope you enjoyed this!

--Team Seeed Studio